In [1]:
import pandas as pd

In [2]:
efficiency_df = pd.read_csv('PLANT TON_EFFICIENCY/separated_23_12.csv')
temperature_df = pd.read_csv('TEMPERATURE/separated_23_12.csv')

In [3]:
columns_to_drop = ['CH1', 'CH2', 'CH3', 'CH4']
efficiency_df.drop(columns=columns_to_drop, inplace=True)

In [4]:
efficiency_df.columns

Index(['Time', 'RT', 'kW_Tot', 'kW_RT', 'CH Load', 'kW_CHH', 'kW_CHP',
       'kW_CHS', 'kW_CDS', 'kW_CT', 'GPM', 'DeltaCHW', 'CHWS', 'CHWR',
       'DeltaCDW', 'CDHI', 'CDLO', 'WBT', 'DeltaCT', 'Hz_ CHP', 'Hz_CHS',
       'Hz_CDS', 'Hz_CT', 'Precent_CH', 'Precent_ CHP', 'Precent_CDS',
       'Precent_CT', 'year', 'month', 'day', 'hour', 'minute', 'second'],
      dtype='object')

In [5]:
temperature_df.columns

Index(['DateTime', 'RH [%]', 'Temperature [°C]', 'WBT_C', 'Time', 'year',
       'month', 'day', 'hour', 'minute', 'second'],
      dtype='object')

In [6]:
efficiency_df.head()

,Time,RT,kW_Tot,kW_RT,CH Load,kW_CHH,kW_CHP,kW_CHS,kW_CDS,kW_CT,...,Precent_CH,Precent_ CHP,Precent_CDS,Precent_CT,year,month,day,hour,minute,second
0,2023-12-01 00:00:00,201.2,208.7,1.037,39.6,135.6,23.3,0,30.9,18.9,...,65.0,11.2,14.8,9.1,2023,12,1,0,0,0
1,2023-12-01 00:10:00,180.5,203.5,1.127,38.4,130.4,23.3,0,30.9,18.9,...,64.1,11.5,15.2,9.3,2023,12,1,0,10,0
2,2023-12-01 00:20:00,170.2,200.8,1.179,37.6,127.8,23.3,0,30.9,18.9,...,63.6,11.6,15.4,9.4,2023,12,1,0,20,0
3,2023-12-01 00:30:00,184.9,201.2,1.088,37.8,128.3,23.3,0,30.6,18.9,...,63.8,11.6,15.2,9.4,2023,12,1,0,30,0
4,2023-12-01 00:40:00,176.4,181.4,1.028,34.6,117.6,24.1,0,30.8,8.9,...,64.8,13.3,17.0,4.9,2023,12,1,0,40,0


In [7]:
temperature_df.head()

,DateTime,RH [%],Temperature [°C],WBT_C,Time,year,month,day,hour,minute,second
0,2023-12-01 00:05:34.000,64.1,30.5,25.1,2023-12-01 00:05:34,2023,12,1,0,5,34
1,2023-12-01 00:15:34.000,64.1,30.4,25.0,2023-12-01 00:15:34,2023,12,1,0,15,34
2,2023-12-01 00:25:34.000,65.1,30.3,25.1,2023-12-01 00:25:34,2023,12,1,0,25,34
3,2023-12-01 00:35:34.000,65.0,30.3,25.1,2023-12-01 00:35:34,2023,12,1,0,35,34
4,2023-12-01 00:45:34.000,64.8,30.3,25.1,2023-12-01 00:45:34,2023,12,1,0,45,34


In [9]:
efficiency_df.shape

(4441, 33)

In [10]:
temperature_df.shape

(4451, 11)

In [11]:
# Efficiency Data
efficiency_df['Time'] = pd.to_datetime(efficiency_df['Time'])

# Temperature Data
temperature_df['DateTime'] = pd.to_datetime(temperature_df['DateTime'])

In [12]:
efficiency_df.sort_values('Time', inplace=True)
temperature_df.sort_values('DateTime', inplace=True)

In [13]:
merged_df = pd.merge_asof(
    efficiency_df,
    temperature_df,
    left_on='Time',
    right_on='DateTime',
    direction='nearest',
    tolerance=pd.Timedelta('6 minutes')  # Adjust as needed
)

In [14]:
# Check for unmatched rows
unmatched_rows = merged_df[merged_df['Temperature [°C]'].isna()]
print(f"Number of unmatched rows: {len(unmatched_rows)}")

Number of unmatched rows: 2


In [16]:
merged_df.shape

(4441, 44)

In [18]:
# Assuming your merged dataframe is called 'merged_df'

# List of temperature columns from df_temperature
temperature_columns = ['Temperature [°C]', 'RH [%]', 'WBT_C']

# Identify rows where any of the temperature columns are NaN
unmatched_rows = merged_df[merged_df[temperature_columns].isna().any(axis=1)]

print(f"Number of unmatched rows: {len(unmatched_rows)}")

Number of unmatched rows: 2


In [19]:
# Remove rows with NaN in any of the temperature columns
merged_df_clean = merged_df.dropna(subset=temperature_columns)

print(f"Rows before cleaning: {len(merged_df)}")
print(f"Rows after cleaning: {len(merged_df_clean)}")

Rows before cleaning: 4441
Rows after cleaning: 4439


In [20]:
# Verify that there are no NaNs in the temperature columns
print(merged_df_clean[temperature_columns].isna().sum())

Temperature [°C]    0
RH [%]              0
WBT_C               0
dtype: int64


In [21]:
merged_df_clean.to_csv('cleaned.csv')

In [23]:
merged_df_clean.columns

Index(['Time_x', 'RT', 'kW_Tot', 'kW_RT', 'CH Load', 'kW_CHH', 'kW_CHP',
       'kW_CHS', 'kW_CDS', 'kW_CT', 'GPM', 'DeltaCHW', 'CHWS', 'CHWR',
       'DeltaCDW', 'CDHI', 'CDLO', 'WBT', 'DeltaCT', 'Hz_ CHP', 'Hz_CHS',
       'Hz_CDS', 'Hz_CT', 'Precent_CH', 'Precent_ CHP', 'Precent_CDS',
       'Precent_CT', 'year_x', 'month_x', 'day_x', 'hour_x', 'minute_x',
       'second_x', 'DateTime', 'RH [%]', 'Temperature [°C]', 'WBT_C', 'Time_y',
       'year_y', 'month_y', 'day_y', 'hour_y', 'minute_y', 'second_y'],
      dtype='object')

In [24]:
# List of columns to drop
columns_to_drop = [
    'Time_y',           # Duplicate time column from temperature data
    'year_x', 'month_x', 'day_x', 'hour_x', 'minute_x', 'second_x',  # Date components from efficiency data
    'year_y', 'month_y', 'day_y', 'hour_y', 'minute_y', 'second_y'   # Date components from temperature data
]

# Drop the columns
merged_df_clean = merged_df_clean.drop(columns=columns_to_drop)

In [25]:
# Rename 'Time_x' to 'Time'
merged_df_clean.rename(columns={'Time_x': 'Time'}, inplace=True)

In [26]:
merged_df_clean.columns

Index(['Time', 'RT', 'kW_Tot', 'kW_RT', 'CH Load', 'kW_CHH', 'kW_CHP',
       'kW_CHS', 'kW_CDS', 'kW_CT', 'GPM', 'DeltaCHW', 'CHWS', 'CHWR',
       'DeltaCDW', 'CDHI', 'CDLO', 'WBT', 'DeltaCT', 'Hz_ CHP', 'Hz_CHS',
       'Hz_CDS', 'Hz_CT', 'Precent_CH', 'Precent_ CHP', 'Precent_CDS',
       'Precent_CT', 'DateTime', 'RH [%]', 'Temperature [°C]', 'WBT_C'],
      dtype='object')

In [27]:
merged_df_clean.to_csv('cleaned.csv')

In [28]:
# Calculate the absolute time difference in seconds
merged_df_clean['Time_Difference'] = (merged_df_clean['Time'] - merged_df_clean['DateTime']).dt.total_seconds().abs()

In [29]:
max_time_diff = merged_df_clean['Time_Difference'].max()
print(f"Maximum Time Difference: {max_time_diff} seconds")

Maximum Time Difference: 353.0 seconds


In [30]:
merged_df_clean.to_csv('cleaned.csv')